## Introduction:

In this notebook, we'll convert the DICOM images in the training set to PNG images. The PNG images will be easier to work with. We can create datasets of varying image sizes for faster prototyping/training.

In [ ]:
!pip install -qU python-gdcm pylibjpeg

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import glob
import gdcm
import pydicom
import matplotlib.pyplot as plt
import random
import shutil
random.seed(123)

from tqdm.notebook import tqdm
from joblib import Parallel, delayed

In [ ]:
DICOM_PATH = '/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images/'
SAVE_FOLDER = '/kaggle/working/train_images_128_png_complete/'
SIZE = 128
EXTENSION = 'png'

os.makedirs(SAVE_FOLDER, exist_ok = True)

In [ ]:
NUM_PATIENTS = len(os.listdir(DICOM_PATH))
print('Number of test patients:', NUM_PATIENTS)

In [ ]:
# def standardize_pixel_array(dcm: pydicom.dataset.FileDataset) -> np.ndarray:
#     """
#     Source : https://www.kaggle.com/competitions/rsna-2023-abdominal-trauma-detection/discussion/427217
#     """
#     # Correct DICOM pixel_array if PixelRepresentation == 1.
#     pixel_array = dcm.pixel_array
#     if dcm.PixelRepresentation == 1:
#         bit_shift = dcm.BitsAllocated - dcm.BitsStored
#         dtype = pixel_array.dtype 
#         pixel_array = (pixel_array << bit_shift).astype(dtype) >>  bit_shift
# #         pixel_array = pydicom.pixel_data_handlers.util.apply_modality_lut(new_array, dcm)

#     intercept = float(dcm.RescaleIntercept)
#     slope = float(dcm.RescaleSlope)
#     center = int(dcm.WindowCenter)
#     width = int(dcm.WindowWidth)
#     low = center - width / 2
#     high = center + width / 2    
    
#     pixel_array = (pixel_array * slope) + intercept
#     pixel_array = np.clip(pixel_array, low, high)

#     return pixel_array

In [ ]:
# def process(patient, size = SIZE, save_folder = "", data_path = ""):
    
#     for study in sorted(os.listdir(data_path + patient)):
#         imgs = {}
#         for f in sorted(glob.glob(data_path + f"{patient}/{study}/*.dcm")):
            
#             dicom = pydicom.dcmread(f)
#             pos_z = dicom[(0x20, 0x32)].value[-1]
            
#             img = standardize_pixel_array(dicom)
#             img = (img - img.min())/(img.max() - img.min() + 1e-6)
            
#             if dicom.PhotometricInterpretation == "MONOCHROME1":
#                 img = 1 - img
            
#             imgs[pos_z] = img
        
#         for i, k in enumerate(sorted(imgs.keys())):
            
#             img = imgs[k]
            
#             img = cv2.resize(img, (size, size))
#             cv2.imwrite(save_folder + f"{patient}_{study}_{i}.png", (img * 255).astype(np.uint8))

In [ ]:
# patients = os.listdir(DICOM_PATH)

In [ ]:
# Parallel(n_jobs = 3)(
#         delayed(process)(patient, size=SIZE, save_folder=SAVE_FOLDER, data_path=TEST_PATH)
#         for patient in tqdm(patients)
#         )

In [ ]:
# random.choice(os.listdir('/kaggle/working/train_images_128_png_complete'))

In [ ]:
# random_file = random.choice(os.listdir(SAVE_FOLDER))
# img = cv2.imread(SAVE_FOLDER + random_file, 0)
# #cv2.imshow('image', img)

# plt.figure(figsize=(15, 15))
# plt.imshow(img, cmap="gray")
# plt.show()

In [ ]:
# !tar -zcf train_images_128_png_complete.tar.gz /kaggle/working/train_images_128_png_complete/

### Create a Smaller DICOM Training Set

For faster prototyping, instead of working with all images for all patients in the training dataset, we'll only a small subset of them. We would like to ensure that all injury labels are present in adequate amount in this small dataset.

In [ ]:
num_patients = 500
num_img = 10
SAVE_PATH = '/kaggle/working/train_images_DICOM_500/'
EXTENSION = 'dcm'
os.makedirs(SAVE_FOLDER, exist_ok = True)

In [ ]:
patients = random.sample(os.listdir(DICOM_PATH), k = num_patients)

for pat in tqdm(patients):
    
    series_id = random.choice(os.listdir(DICOM_PATH + str(pat)))
    SOURCE_FOLDER = DICOM_PATH + pat + '/' + series_id + '/'
    TARGET_FOLDER = SAVE_PATH + pat + '/'
    os.makedirs(TARGET_FOLDER, exist_ok = True)
    
    files_to_copy = random.choices(os.listdir(SOURCE_FOLDER), k = num_img)
    
    for file in files_to_copy:
        file_path = os.path.join(SOURCE_FOLDER + file)
        shutil.copy(file_path, TARGET_FOLDER)
    

Let us look at the distribution of labels in this small DICOM image set.

In [ ]:
patient_labels = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/train.csv')
patients = [int(pat) for pat in patients]
x = patient_labels[patient_labels['patient_id'].isin(patients)]
x

In [ ]:
for pat in patients:
    if pat not in list(patient_labels['patient_id']):
        print(pat)

In [ ]:
len(set(patients))

In [ ]:
def get_pos_num(df, label):
    num_entries = df.shape[0]
    return (df[label] == 1).sum()

injury_labels = ['bowel_injury', 'extravasation_injury' , 'kidney_low' , 'kidney_high', 'liver_low',
                 'liver_high', 'spleen_low', 'spleen_high']

print(f'Total number of patients: {num_patients}')
for label in injury_labels:    
    print(f'Number of patients with {label} = {get_pos_num(patient_labels, label)}')

In [ ]:
!du -sh /kaggle/working/train_images_DICOM_500/

In [ ]:
!rm -r /kaggle/working/train_images_128_png_complete